In [1]:
from tp3 import *
import numpy as np
import matplotlib.pyplot as plt

# 1. Matemathical Model

In [73]:
def get_pos(angles, positions):
    A = np.column_stack([np.cos(angles), -np.sin(angles)])    # Design matrix
    b = np.sum(A * positions, axis = 1) # Solutions for linear system
    At = A.T
    return np.linalg.inv(At @ A) @ At @ b # Least Squares solution    

# 2. Simulation

In [74]:
# Import config_1

theta_1 = np.array([config_1[p]['angulo'] for p in config_1.keys()])
pos_1 = np.array([config_1[p]['pos'] for p in config_1.keys()])

x = get_pos(theta_1, pos_1)
print(x)

[100.  50.]


In [75]:
# Gaussian noise
rng = np.random.default_rng(42) # Random generator for reprodoctubility

theta_1 = np.array([config_1[p]['angulo'] for p in config_1.keys()])
pos_1 = np.array([config_1[p]['pos'] for p in config_1.keys()])

# Introduce noise
noise = np.array([rng.normal(0, sigma, size=2) for sigma in (2, 5, 10)])
pos_1 += noise

# Now everything again
x = get_pos(theta_1, pos_1)
print(x)

[100.05916277  51.8356695 ]
